In [1]:
import os

import arviz as az
import bambi as bmb
import pandas as pd
import numpy as np

from arviz.plots.plot_utils import calculate_point_estimate as calc_point_est

## Helper functions

In [2]:
def is_inside(val, interval):
    """
    Check if the value `val` is inside the interval `interval`.
    """
    if val >= interval[0] and val <= interval[1]:
        return True
    else:
        return False

def get_pval_decision(pval, alpha=0.05):
    if pval <= alpha:
        return "reject H0"
    else:
        return "fail to reject H0"

def get_ci_decision(val, interval):
    if is_inside(val, interval):
        return "fail to reject H0"
    else:
        return "reject H0"


def get_bf_decision(bfA0, cutoff_reject_H0=3, cutoff_accept_H0=1/3):
    if bfA0 >= cutoff_reject_H0:
        return "reject H0"
    elif bfA0 <= cutoff_accept_H0:
        return "accept H0"
    else:
        return "no decision"


def calc_dmeans(idata, group_name="group", groups=["ctrl", "treat"]):
    """
    Simplified version of `ministats.bayes.calc_dmeans_stats`
    """
    group_dim = group_name + "_dim"
    post = idata["posterior"]
    post["mu_" + groups[0]] = post[group_name].loc[{group_dim:groups[0]}]
    post["mu_" + groups[1]] = post[group_name].loc[{group_dim:groups[1]}]
    post["dmeans"] = post["mu_" + groups[1]] - post["mu_" + groups[0]]

## Sensitivity analysis

In [3]:
# Original (for testing)
iqs2 = pd.read_csv("../datasets/iqs2.csv")

In [4]:
from ministats.book.tables import fit_bayesian_model_iqs2
from ministats.bayes import calc_dmeans_stats

idata2 = fit_bayesian_model_iqs2(iqs2, {}, random_seed=42)
calc_dmeans_stats(idata2, group_name="group")
post2 = idata2["posterior"]
summary2 = az.summary(post2, kind="stats", hdi_prob=0.95)
summary2

,mean,sd,hdi_2.5%,hdi_97.5%
group[ctrl],100.522,0.211,100.110,100.934
group[treat],101.545,0.380,100.798,102.305
nu,1.850,0.485,1.018,2.802
sigma_group[ctrl],0.006,0.196,-0.378,0.382
sigma_group[treat],0.673,0.211,0.230,1.063
mu_ctrl,100.522,0.211,100.110,100.934
mu_treat,101.545,0.380,100.798,102.305
dmeans,1.022,0.436,0.179,1.895
sigma_ctrl,1.026,0.202,0.649,1.414
sigma_treat,2.004,0.422,1.233,2.844


In [5]:
from ministats.book.tables import sens_analysis_dmeans_iqs2
results = sens_analysis_dmeans_iqs2(iqs2)
results

,M_prior,logSigma_prior,Nu_prior,dmeans_mean,dmeans_95hdi,dsigmas_mode,dsigmas_95hdi,nu_mode,codhend_mode
0,"$\mathcal{N}(100,35)$","$\mathcal{N}(1,2)$","$\Gamma(2,0.1)$",1.022,"[0.179, 1.895]",0.923,"[0.176, 1.868]",1.649,0.923
1,"$\mathcal{N}(100,50)$","$\mathcal{N}(1,2)$","$\Gamma(2,0.1)$",1.023,"[0.201, 1.895]",0.926,"[0.199, 1.857]",1.69,0.926
2,"$\mathcal{N}(100,10)$","$\mathcal{N}(1,2)$","$\Gamma(2,0.1)$",1.023,"[0.212, 1.93]",0.836,"[0.163, 1.821]",1.675,0.836
3,"$\mathcal{N}(100,35)$","$\mathcal{N}(0,1)$","$\Gamma(2,0.1)$",1.016,"[0.18, 1.832]",0.837,"[0.176, 1.797]",1.609,0.837
4,"$\mathcal{N}(100,35)$","$\mathcal{N}(1,2)$",$\textrm{Expon}(1/30)$,1.027,"[0.213, 1.856]",0.912,"[0.162, 1.808]",1.601,0.912


In [6]:
# display_cols = ['M_prior', 'logSigma_prior', 'Nu_prior', 'dmeans_mean', 'dmeans_95hdi', 'dsigmas_mode', 'nu_mode']
# with pd.option_context('styler.format.precision', 3):
#     print(results[display_cols].to_latex(index=False))

## Performance analysis

### Dataset

In [7]:
# # Original (for testing)
# iqs2 = pd.read_csv("../datasets/iqs2.csv")
# treated = iqs2[iqs2["group"]=="treat"]["iq"].values
# controls = iqs2[iqs2["group"]=="ctrl"]["iq"].values
# dataset = iqs2


In [8]:
from ministats.book.tables import gen_dmeans_dataset

dataset = gen_dmeans_dataset(n=50, Delta=0.4, prop_outliers=0.05, random_seed=45)
treated = dataset[dataset["group"]=="treat"]["value"].values
controls = dataset[dataset["group"]=="ctrl"]["value"].values

print(treated[0:20])
print(controls[0:20])
# dataset.groupby("group").head()

[ 3.38666824  4.05043186 -0.19195943 -0.50781168  0.67179962  0.39543833
  1.25410694 -1.19000488  0.45604899  0.50586257  0.30916242 -0.31387606
  0.60371539  1.60303184  0.48431307  0.56393392  0.77375155  0.17309315
 -0.2712744   0.71410988]
[ 0.48252047  2.14383013 -0.39514554 -0.20430091 -1.27163265 -2.59687863
  0.28968091 -0.87330464  0.39407266  0.93510554 -0.01568471  0.25959597
 -1.47331424  0.8019266  -1.75075239 -0.49505193 -1.00860081  0.02524419
 -0.12150685 -1.54687318]


In [9]:
from ministats.book.tables import fit_dmeans_models

results = fit_dmeans_models(dataset, random_seed=42)

In [10]:
from ministats.book.tables import gen_dmeans_datasets
gen_dmeans_datasets(
    ns=[20,30,50,100],
    Deltas=[0, 0.2, 0.5, 0.8, 1.3],
    outliers_options=["no", "few", "lots"],
    random_seed_start=45)

[{'n': 20, 'Delta': 0, 'outliers': 'no', 'random_seed': 45},
 {'n': 20, 'Delta': 0, 'outliers': 'few', 'random_seed': 46},
 {'n': 20, 'Delta': 0, 'outliers': 'lots', 'random_seed': 47},
 {'n': 20, 'Delta': 0.2, 'outliers': 'no', 'random_seed': 48},
 {'n': 20, 'Delta': 0.2, 'outliers': 'few', 'random_seed': 49},
 {'n': 20, 'Delta': 0.2, 'outliers': 'lots', 'random_seed': 50},
 {'n': 20, 'Delta': 0.5, 'outliers': 'no', 'random_seed': 51},
 {'n': 20, 'Delta': 0.5, 'outliers': 'few', 'random_seed': 52},
 {'n': 20, 'Delta': 0.5, 'outliers': 'lots', 'random_seed': 53},
 {'n': 20, 'Delta': 0.8, 'outliers': 'no', 'random_seed': 54},
 {'n': 20, 'Delta': 0.8, 'outliers': 'few', 'random_seed': 55},
 {'n': 20, 'Delta': 0.8, 'outliers': 'lots', 'random_seed': 56},
 {'n': 20, 'Delta': 1.3, 'outliers': 'no', 'random_seed': 57},
 {'n': 20, 'Delta': 1.3, 'outliers': 'few', 'random_seed': 58},
 {'n': 20, 'Delta': 1.3, 'outliers': 'lots', 'random_seed': 59},
 {'n': 30, 'Delta': 0, 'outliers': 'no', 'rand

In [11]:
from ministats.book.tables import calc_dmeans_perf_metrics
results = calc_dmeans_perf_metrics(
    ns=[20,30,50,100],
    Deltas=[0, 0.2, 0.5, 0.8, 1.3],
    outliers_options=["no", "few", "lots"],
    reps=100)
results

Simulating a total of 60 dataset specs
loaded cached results from  simdata/dmeans_perf_metrics__ns_20_30_50_100__Deltas_0_0.2_0.5_0.8_1.3__outs_no_few_lots__reps_100.csv


n  Delta outliers  seed  count_reject  \
spec model                                                   
0    perm           20    0.0       no    45             6   
     welch          20    0.0       no    45             6   
     norm_bayes     20    0.0       no    45             2   
     robust_bayes   20    0.0       no    45             7   
     bf             20    0.0       no    45             2   
...                ...    ...      ...   ...           ...   
59   perm          100    1.3     lots   104           100   
     welch         100    1.3     lots   104           100   
     norm_bayes    100    1.3     lots   104           100   
     robust_bayes  100    1.3     lots   104           100   
     bf            100    1.3     lots   104            99   

                   count_fail_to_reject  count_captured  avg_width  
spec model                                                          
0    perm                            94            90.0   1.011714  
     welch                           94            90.0   1.065525  
     norm_bayes                      98            94.0   1.294730  
     robust_bayes                    93            91.0   1.079160  
     bf                              98             NaN        NaN  
...                                 ...             ...        ...  
59   perm                             0            90.0   0.678436  
     welch                            0            90.0   0.685557  
     norm_bayes                       0            91.0   0.686790  
     robust_bayes                     0            94.0   0.504730  
     bf                               1             NaN        NaN  

[300 rows x 8 columns]

In [12]:
from ministats.book.tables import get_perf_table_typeI

tableA = get_perf_table_typeI(results)
tableA
# print(tableA.to_latex(float_format="%.2f"))

model         perm  welch  norm_bayes  robust_bayes    bf
outliers n                                               
no       20   0.06   0.06        0.02          0.07  0.02
         30   0.04   0.04        0.03          0.04  0.03
         50   0.04   0.04        0.01          0.04  0.01
         100  0.06   0.06        0.03          0.05  0.01
few      20   0.04   0.04        0.01          0.05  0.01
         30   0.04   0.04        0.01          0.05  0.00
         50   0.03   0.03        0.02          0.01  0.00
         100  0.07   0.07        0.06          0.08  0.03
lots     20   0.04   0.02        0.01          0.03  0.01
         30   0.02   0.02        0.00          0.02  0.00
         50   0.07   0.06        0.06          0.07  0.04
         100  0.03   0.03        0.03          0.02  0.00

In [13]:
from ministats.book.tables import get_perf_table_power

tableB = get_perf_table_power(results, show_all=False)
tableB
# tableB_str = tableB.to_latex(float_format="%.2f")
# print(tableB_str.replace("0.500000", "0.5").replace("0.800000","0.8"))

model               perm  welch  norm_bayes  robust_bayes    bf
outliers Delta n                                               
no       0.5   30   0.46   0.46        0.33          0.46  0.31
               50   0.68   0.69        0.54          0.66  0.50
               100  0.93   0.93        0.90          0.93  0.84
         0.8   20   0.60   0.60        0.44          0.59  0.49
               30   0.86   0.86        0.77          0.86  0.72
               50   0.99   0.99        0.96          0.99  0.93
few      0.5   30   0.49   0.49        0.32          0.45  0.29
               50   0.49   0.51        0.39          0.58  0.33
               100  0.80   0.80        0.80          0.88  0.69
         0.8   20   0.69   0.68        0.51          0.65  0.54
               30   0.88   0.88        0.79          0.87  0.78
               50   0.85   0.85        0.83          0.94  0.75
lots     0.5   30   0.33   0.32        0.29          0.43  0.20
               50   0.52   0.52        0.50          0.63  0.33
               100  0.64   0.63        0.60          0.88  0.52
         0.8   20   0.44   0.43        0.37          0.63  0.30
               30   0.60   0.59        0.57          0.81  0.48
               50   0.82   0.83        0.79          0.97  0.66

In [14]:
from ministats.book.tables import get_perf_table_coverage

tableC = get_perf_table_coverage(results)
tableC
# print(tableC.to_latex(float_format="%.2f"))

model            perm              welch           norm_bayes            \
             coverage avg_width coverage avg_width   coverage avg_width   
outliers n                                                                
no       20    0.8800  1.011711   0.9050  1.064862     0.9650  1.298070   
         30    0.9100  0.821001   0.9175  0.849298     0.9625  1.016220   
         50    0.9050  0.651732   0.9100  0.665610     0.9525  0.762513   
         100   0.9025  0.462228   0.9025  0.467091     0.9325  0.517568   
few      20    0.8550  1.002797   0.8675  1.056252     0.9500  1.294572   
         30    0.9175  0.835295   0.9250  0.864310     0.9625  1.022407   
         50    0.8675  0.784709   0.9025  0.802027     0.9350  0.854340   
         100   0.8725  0.554332   0.8975  0.560776     0.9125  0.581300   
lots     20    0.8700  1.442238   0.9025  1.530370     0.9450  1.601472   
         30    0.8550  1.085291   0.8700  1.128444     0.9000  1.197037   
         50    0.8825  0.892523   0.8975  0.913113     0.9175  0.937645   
         100   0.8975  0.679404   0.9025  0.686951     0.9175  0.688840   

model        robust_bayes            
                 coverage avg_width  
outliers n                           
no       20        0.9100  1.078473  
         30        0.9225  0.855460  
         50        0.9100  0.667750  
         100       0.9075  0.467747  
few      20        0.8700  1.069132  
         30        0.9250  0.872108  
         50        0.9075  0.695260  
         100       0.9275  0.487010  
lots     20        0.8975  1.197020  
         30        0.8550  0.926892  
         50        0.9075  0.726270  
         100       0.8950  0.509295

In [15]:
# results[(results["outliers"]=="few") & (results["Delta"]==1.0)]

In [16]:
# results[(results["outliers"]=="lots") & (results["Delta"]==1.0)]


### Permutation test

In [17]:
from ministats.hypothesis_tests import permutation_test_dmeans
from ministats.confidence_intervals import ci_dmeans

# np.random.seed(random_seed)
pval = permutation_test_dmeans(treated, controls)
print(pval)
decision = get_pval_decision(pval)
ci90 = ci_dmeans(treated, controls, alpha=0.1, method="b")
decision, ci90

0.0006


('reject H0', [0.41048098294960556, 1.1117862568974726])

### Welch's two-sample t-test

In [18]:
from ministats.hypothesis_tests import ttest_dmeans

pval = ttest_dmeans(treated, controls, equal_var=False, alt="two-sided")
print(pval)
decision = get_pval_decision(pval)
ci90 = ci_dmeans(treated, controls, alpha=0.1, method="a")
decision, ci90

0.0005822820572150585


('reject H0', [0.3997722238319435, 1.100183880609041])

### Normal Bayesian model

In [19]:
random_seed = 43

from ministats.bayes import hdi_from_idata

priors = {
    "group": bmb.Prior("Normal", mu=0, sigma=2),
    "sigma": {
        "group": bmb.Prior("LogNormal", mu=0, sigma=1)
        # "group": bmb.Prior("Normal", mu=1, sigma=1)
    }
}
formula = bmb.Formula("value ~ 0 + group", "sigma ~ 0 + group")
norm_mod = bmb.Model(formula=formula, family="gaussian", priors=priors, data=dataset)
idata = norm_mod.fit(random_seed=random_seed)
calc_dmeans(idata)
ci95 = hdi_from_idata(idata, var_name="dmeans", hdi_prob=0.95)
print("ci95 =", ci95)
decision = get_ci_decision(0, ci95)
ci90 = hdi_from_idata(idata, var_name="dmeans", hdi_prob=0.9)
decision, ci90

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [group, sigma_group]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 1 seconds.


ci95 = [0.281, 1.2]


('reject H0', [0.354, 1.142])

### Robust Bayesian model

In [20]:
random_seed = 42

from ministats.bayes import hdi_from_idata


priors = {
    "group": bmb.Prior("Normal", mu=0, sigma=2),
    "sigma": {
        "group": bmb.Prior("Normal", mu=0, sigma=1)
    },
    "nu": bmb.Prior("Gamma", alpha=2, beta=0.1),
}
formula = bmb.Formula("value ~ 0 + group", "sigma ~ 0 + group")
robust_mod = bmb.Model(formula=formula, family="t", priors=priors, data=dataset)
idata = robust_mod.fit(random_seed=random_seed)
calc_dmeans(idata)

nus = idata["posterior"]["nu"].values.flatten()
nu_mode = calc_point_est("mode", nus).round(3)
print("nu_mode =", nu_mode)

ci95 = hdi_from_idata(idata, var_name="dmeans", hdi_prob=0.95)
decision = get_ci_decision(0, ci95)
ci90 = hdi_from_idata(idata, var_name="dmeans", hdi_prob=0.9)

decision, ci90

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, group, sigma_group]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 2 seconds.


nu_mode = 5.987


('reject H0', [0.39, 1.062])

### Bayes factors

In [21]:
# Computer Bayes factor (based on JZS prior)
from scipy.stats import ttest_ind
import pingouin as pg

ttres = ttest_ind(treated, controls, equal_var=True)
tstat = ttres.statistic
n = len(treated)
m = len(controls)
bfA0 = pg.bayesfactor_ttest(tstat, nx=n, ny=m, r=0.707)
print(bfA0)
decision = get_bf_decision(bfA0)
ci90 = None
decision, ci90

47.19376683007691


('reject H0', None)

In [22]:
# # Verify using R
# import rpy2.robjects as robjects
# from rpy2.robjects import r as runr, pandas2ri
# from rpy2.robjects.packages import importr
# pandas2ri.activate()
# # import the BayesFactor package
# BayesFactor = importr('BayesFactor')
# # copy the data sample into the R workspace
# robjects.globalenv["dataset"] = dataset
# # compute the Bayes factor
# runr('bf = ttestBF(formula = value ~ group, data = dataset)')
# runr('print(bf)')